# TF_Record

- 데이터가 이미지에 비해 상대적으로 적은 데이터는, 메모리에 데이터가 올라가,
   일반적으로는 TF_Record를 쓸 필요가 없습니다.
   
   
- 하지만 대용량의 데이터의 경우는 TF_record를 사용하는 것이 load, save, train등에
   영향을 미치기 때문에 권장하는 바입니다.
  

- Tf_record로 데이터를 치환하기 위해서는 Dictionary 형태로 치환

In [29]:
import tensorflow as tf

import numpy as np

import pandas as pd

import tempfile


In [21]:
text = '안녕하세요, 저는 신성진 입니다.'
text_idx = np.random.randint(10, size=10)
text_label = 1

In [23]:
text_dict = {'text': text, 'text_idx': text_idx, 'text_label': text_label}

In [40]:
sequences = [[1, 2, 3], [4, 5, 1], [1, 2]]
label_sequences = [[0, 1, 0], [1, 0, 0], [1, 1]]

In [41]:
def make_example(sequence, labels):
    
    # The obj we return
    ex = tf.train.SequenceExample()
    sequence_length = len(sequence) # sequence 정보가 아닌 내용
    ex.context.feature["length"].int64_list.value.append(sequence_length)
    
    # two feature sequences, feature_lists가 뭘 의미하지?
    fl_tokens = ex.feature_lists.feature_list["token"]
    fl_labels = ex.feature_lists.feature_list["labels"]
    
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
        
    return ex

In [45]:
with tempfile.NamedTemporaryFile() as fp:
    writer = tf.python_io.TFRecordWriter(fp.name)
    for sequence, label_sequence in zip(sequences, label_sequences):
        print(sequence, label_sequence)
        ex = make_example(sequence, label_sequence)
        writer.write(ex.SerializeToString())
    writer.close()
    print("Wrote to {}".format(fp.name))

[1, 2, 3] [0, 1, 0]
[4, 5, 1] [1, 0, 0]
[1, 2] [1, 1]
Wrote to /var/folders/cp/9zlspqj13_j36gnd9q2yp0180000gn/T/tmpsr4hfg67


In [47]:
# Parsing
tf.reset_default_graph()

# single serialized 예
ex = make_example([1,2,3], [0,1,0]).SerializeToString()

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}

sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64),
}

In [48]:
#https://medium.com/@TalPerry/getting-text-into-tensorflow-with-the-dataset-api-ffb832c8bec6
#http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/

#Parsing
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=ex,
    context_features=context_features,
    sequence_features=sequence_features
)

In [54]:
tf_record_data='/var/folders/cp/9zlspqj13_j36gnd9q2yp0180000gn/T/tmpsr4hfg67'

dataset = tf.data.TFRecordDataset([tf_record_data])
a = dataset.make_one_shot_iterator()
b = a.get_next()

In [60]:
sess = tf.InteractiveSession()
sess.run(b)

NotFoundError: /var/folders/cp/9zlspqj13_j36gnd9q2yp0180000gn/T/tmpsr4hfg67; No such file or directory
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[]], output_types=[DT_STRING], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_2)]]

Caused by op 'IteratorGetNext', defined at:
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/user/tf14_py_3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/user/tf14_py_3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/user/tf14_py_3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486,

In [52]:
# tf.contrib.learn.run_n: Run output_dict tensors n times, with the same feed_dict
# 테스트용이고, estimator가 나오면서, 기능 지원을 안하기 때문에 참고용

context = tf.contrib.learn.run_n(context_parsed, n=1, feed_dict=None)
print(context[0])
context = tf.contrib.learn.run_n(sequence_parsed, n=1, feed_dict=None)
print(sequence[0])

InvalidArgumentError: Name: , Feature list 'tokens' is required but could not be found.  Did you mean to include it in feature_list_dense_missing_assumed_empty or feature_list_dense_defaults?
	 [[Node: ParseSingleSequenceExample/ParseSingleSequenceExample = ParseSingleSequenceExample[Ncontext_dense=1, Ncontext_sparse=0, Nfeature_list_dense=2, Nfeature_list_sparse=0, Tcontext_dense=[DT_INT64], context_dense_shapes=[[]], context_sparse_types=[], feature_list_dense_shapes=[[], []], feature_list_dense_types=[DT_INT64, DT_INT64], feature_list_sparse_types=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"](ParseSingleSequenceExample/ParseSingleSequenceExample/serialized, ParseSingleSequenceExample/ParseSingleSequenceExample/feature_list_dense_missing_assumed_empty, ParseSingleSequenceExample/ParseSingleSequenceExample/context_dense_keys_0, ParseSingleSequenceExample/ParseSingleSequenceExample/feature_list_dense_keys_0, ParseSingleSequenceExample/ParseSingleSequenceExample/feature_li

In [24]:
for (data, path) in [(text_dict, './text_dict.tfrecord')]:
    with open(paht,'w') as f:
        writer = tf.python_io.TFRecordWriter(f.name)

    ex = tf.train.SequenceExample()
    ex.context.feature['label'].int64_list.value.append(10 + 2) # +2 for start and end
    ex.context.feature['text_idx'].int64_list.value.append(text_idx)
    
    for example in data:
        
        
        record = 
    

{'text': '안녕하세요, 저는 신성진 입니다.',
 'text_idx': array([6, 8, 8, 6, 4, 1, 9, 2, 6, 4]),
 'text_label': 1}